In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE142153'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Human PBMCs: Healthy vs Diabetic nephropathy vs ESRD"
!Series_summary	"Transcriptional profiling of human PBMCs comparing healthy controls, patients with diabetic nephropathy and patients with ESRD. PBMCs were analyzed as they mediate inflammatory injury. Goal was to determine effects of increasing severity of diabetic nephropathy on global PBMC gene expression. Microarray analysis of PBMCs taken from patients with varying degrees of diabetic nephropathy."
!Series_overall_design	"3 condition experiment - Healthy control (10) vs diabetic nephropathy (23) vs ESRD (7)"
Sample Characteristics Dictionary:
{0: ['tissue: peripheral blood'], 1: ['diagnosis: healthy control', 'diagnosis: diabetic nephropathy', 'diagnosis: ESRD']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False  # Change to True if gene expression data is available
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset likely contains gene expression data
is_gene_available = True

# Data Availability section
trait_row = 1
age_row = None
gender_row = None

# Data Type Conversion section

def convert_trait(value):
    try:
        trait_map = {
            'healthy control': 0,
            'diabetic nephropathy': 1,
            'ESRD': 2
        }
        return trait_map.get(value.split(':')[-1].strip(), None)
    except:
        return None

# Function to save cohort information
save_cohort_info('GSE142153', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction section
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE142153.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM4221568': [0], 'GSM4221569': [0], 'GSM4221570': [0], 'GSM4221571': [0], 'GSM4221572': [0], 'GSM4221573': [0], 'GSM4221574': [0], 'GSM4221575': [0], 'GSM4221576': [0], 'GSM4221577': [0], 'GSM4221578': [1], 'GSM4221579': [1], 'GSM4221580': [1], 'GSM4221581': [1], 'GSM4221582': [1], 'GSM4221583': [1], 'GSM4221584': [1], 'GSM4221585': [1], 'GSM4221586': [1], 'GSM4221587': [1], 'GSM4221588': [1], 'GSM4221589': [1], 'GSM4221590': [1], 'GSM4221591': [1], 'GSM4221592': [1], 'GSM4221593': [1], 'GSM4221594': [1], 'GSM4221595': [1], 'GSM4221596': [1], 'GSM4221597': [1], 'GSM4221598': [1], 'GSM4221599': [1], 'GSM4221600': [1], 'GSM4221601': [2], 'GSM4221602': [2], 'GSM4221603': [2], 'GSM4221604': [2], 'GSM4221605': [2], 'GSM4221606': [2], 'GSM4221607': [2]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100111', 'A_23_P100127', 'A_23_P100133', 'A_23_P100141',
       'A_23_P100156', 'A_23_P100189', 'A_23_P100203', 'A_23_P100220',
       'A_23_P100240', 'A_23_P10025', 'A_23_P100278', 'A_23_P100292',
       'A_23_P100315', 'A_23_P100344', 'A_23_P100355', 'A_23_P100386'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. When analyzing a gene expression dataset, we need to map some identifiers of genes to actual gene symbols.

identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df) 


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Chronic_kidney_disease/gene_data/GSE142153.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Chronic_kidney_disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE142153', './preprocessed/Chronic_kidney_disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Chronic_kidney_disease/GSE142153.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Chronic_kidney_disease':
  25%: 0.75
  50% (Median): 1.0
  75%: 1.0
Min: 0.0
Max: 2.0
The distribution of the feature 'Chronic_kidney_disease' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Chronic_kidney_disease/cohort_info.json
